In [89]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [90]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [91]:
movies = pd.read_csv('movies.csv')
movies.head(2)

,title,original_title,tagline,overview,genres,cast,director,keywords,original_language,popularity,...,production_countries,release_date,revenue,runtime,spoken_languages,status,budget,vote_average,vote_count,homepage
0,Avatar,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",James Cameron,"['culture clash', 'future', 'space war', 'spac...",en,150.437577,...,"['United States of America', 'United Kingdom']",2009-12-10,2787965087,162.0,"['English', 'Español']",Released,237000000,7.2,11800,http://www.avatarmovie.com/
1,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",Gore Verbinski,"['ocean', 'drug abuse', 'exotic island', 'east...",en,139.082615,...,['United States of America'],2007-05-19,961000000,169.0,['English'],Released,300000000,6.9,4500,http://disney.go.com/disneypictures/pirates/


In [92]:
required_columns =['title','original_title', 'tagline', 'keywords', 'overview', 'genres', 'cast', 'director']
movies = movies[required_columns]
movies.head(n=3)
print(movies.isna().sum())

,title,original_title,tagline,keywords,overview,genres,cast,director
0,Avatar,Avatar,Enter the World of Pandora.,"['culture clash', 'future', 'space war', 'spac...","In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",James Cameron
1,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","['ocean', 'drug abuse', 'exotic island', 'east...","Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",Gore Verbinski
2,Spectre,Spectre,A Plan No One Escapes,"['spy', 'based on novel', 'secret agent', 'seq...",A cryptic message from Bond’s past sends him o...,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",Sam Mendes


title               0
original_title      0
tagline           844
keywords            0
overview            3
genres              0
cast                0
director           30
dtype: int64


In [93]:
movies.fillna(' ', inplace=True)
print(movies.isna().sum())
print(movies.iloc[0])

title             0
original_title    0
tagline           0
keywords          0
overview          0
genres            0
cast              0
director          0
dtype: int64
title                                                        Avatar
original_title                                               Avatar
tagline                                 Enter the World of Pandora.
keywords          ['culture clash', 'future', 'space war', 'spac...
overview          In the 22nd century, a paraplegic Marine is di...
genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
director                                              James Cameron
Name: 0, dtype: object


In [94]:
# Create movie CONTENT
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies['director'].apply(lambda x: x.replace(" ","") )

In [95]:
# Converting List to Strings
movies['genres'] = movies['genres'].apply(lambda x: ','.join(map(str, x)))
movies['keywords'] = movies['keywords'].apply(lambda x: ','.join(map(str, x)))
movies['cast'] = movies['cast'].apply(lambda x: ','.join(map(str, x)))

In [96]:
movies['content'] = movies['title'] + ' ' + movies['overview'] + ' ' + movies['keywords'] + ' ' + movies['cast'] + ' ' + movies ['director']
print(movies['content'])

0       Avatar In the 22nd century, a paraplegic Marin...
1       Pirates of the Caribbean: At World's End Capta...
2       Spectre A cryptic message from Bond’s past sen...
3       The Dark Knight Rises Following the death of D...
4       John Carter John Carter is a war-weary, former...
                              ...                        
4804    El Mariachi El Mariachi just wants to play his...
4805    Newlyweds A newlywed couple's honeymoon is upe...
4806    Signed, Sealed, Delivered "Signed, Sealed, Del...
4807    Shanghai Calling When ambitious New York attor...
4808    My Date with Drew Ever since the second grade ...
Name: content, Length: 4809, dtype: object


In [108]:
vectorizer = TfidfVectorizer(max_features=1000)
movie_vectors = vectorizer.fit_transform(movies['content'].values) 

In [109]:
similarity = cosine_similarity(movie_vectors)
similarity_df = pd.DataFrame(similarity)

In [111]:
def recommend(movie):
    # find movie index from dataset
    movies_index = movies[movies['title'] == movie].index[0]
    # finding cosine similarities of movie
    distances = similarity[movies_index]
    move_train=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])
    movies_list = move_train[1:10]
    result=[]
    for i in movies_list:
        entity=movies.iloc[i[0]].title
        result.append(entity)
    return result


In [112]:
# print(movies.title.head(10))
# print('')
a=recommend('Circumstance')
print(a)

['Slums of Beverly Hills', 'The Loss of Sexual Innocence', '16 to Life', 'Indignation', 'Fifty Shades of Black', 'Furious 7', 'Halloween II', 'A Better Life', 'Maria Full of Grace']
